# [1] Vehicle API Tutorial

## 1.1. Vehicle Client

In [ ]:
import os
from dotenv import load_dotenv
from aicastle.deepracer.vehicle.api.client import VehicleClient
load_dotenv(".env")

vehicle =VehicleClient(
    ip=os.getenv("VEHICLE_IP"),
    password=os.getenv("VEHICLE_PASSWORD"),
)


### 1.1.1. info 속성

In [ ]:
print(vehicle.info)

### 1.1.2. frame_queue

In [ ]:
print("frame_queue length: ", len(vehicle.frame_queue))
print("frame_shape: ", vehicle.frame_queue[0].shape)
# print("frame_queue last: \n", vehicle.frame_queue[-1])

### 1.1.3. show_image

In [ ]:
from aicastle.deepracer.vehicle.api.utils import show_image

show_image(
    frame_queue = vehicle.frame_queue,
    window_name=vehicle.ip
)

### 1.1.4. move

In [ ]:
import time

vehicle.move(angle=0, speed=3)
time.sleep(1)
vehicle.move(angle=0, speed=0)

### 1.1.5. set_speed_percent

In [ ]:
import time

vehicle.set_speed_percent(70)
vehicle.move(angle=0, speed=3)
time.sleep(1)

vehicle.set_speed_percent(50)
vehicle.move(angle=0, speed=3)
time.sleep(1)

vehicle.move(angle=0, speed=0)

### 1.1.6. speed_percent up down

In [ ]:
vehicle.speed_percent_increase = 1

print("Origin: ", vehicle.info["speed_percent"])

vehicle.speed_percent_up()
print("After speed up: ", vehicle.info["speed_percent"])

vehicle.speed_percent_down()
print("After speed down: ", vehicle.info["speed_percent"])


### 1.1.7. uploaded_model_list

In [ ]:
vehicle.uploaded_model_list()

### 1.1.8. upload_model

In [ ]:
model_zip_path = "saved_data/models/model2.tar.gz"

model_name = vehicle.upload_model(model_zip_path)
print("Model name: ", model_name)

### 1.1.9. load_model

- load_model

In [ ]:
model_name = "model1"

vehicle.load_model(model_name)

- stop

In [ ]:
vehicle.stop()

- set speed percent 70

In [ ]:
vehicle.set_speed_percent(70)

- set speed percent 50

In [ ]:
vehicle.set_speed_percent(50)

- stop

In [ ]:
vehicle.stop()

### 1.1.10. calibration

- get_calibration_angle

In [ ]:
vehicle.get_calibration_angle()

- get_calibration_speed

In [ ]:
vehicle.get_calibration_speed()

- set_calibration_angle

In [ ]:
vehicle.set_calibration_angle(center=-5, left=25, right=-35)

- set_calibration_speed

In [ ]:
vehicle.set_calibration_speed(stopped=-12, forward=-50, backward=20)

- calibration result test

In [ ]:
vehicle.move_forward(duration=1)
# vehicle.move_left(duration=1)
# vehicle.move_right(duration=1)

## 1.2. Keyboard Listener

### 1.2.1. keyboard_listner

In [ ]:
from aicastle.deepracer.vehicle.api.utils import KeyboardListener

keyboard_listener = KeyboardListener(
    on_press_callback={
        "+": [vehicle.speed_percent_up],
        "-": [vehicle.speed_percent_down],
        "Key.enter": [vehicle.start],
        "Key.space": [vehicle.stop],
    }
)

- info

In [ ]:
print(keyboard_listener.info)

### 1.2.2. speed_percent up down test

- 아래 코드 실행 후 `+`, `-` 누르면서 결과 확인하기

In [ ]:
vehicle.move(angle=0, speed=3)

- 정지

In [ ]:
vehicle.stop()

### 1.2.3. auto mode start stop test

- 아래 코드 실행 후 `enter`, `space`, `+`, `-` 누르면서 결과 확인하기

In [ ]:
model_name = "model1"

vehicle.load_model(model_name)

## 1.3. Show Info

### 1.3.1. show_info

In [ ]:
from aicastle.deepracer.vehicle.api.utils import show_info

show_info_stop_event = show_info(
    info_containers=[
        keyboard_listener.info,
        vehicle.info,
    ],
    show_keys=[
        "pressed_key",
        "battery_level",
        "speed_percent",
        "fetch_fps",

    ]
)

### 1.3.2. show_info_stop

In [ ]:
# show_info_stop_event.set()

## 1.4. Keyboard Controller

- set controller

In [ ]:
from aicastle.deepracer.vehicle.api.controller import KeyboardController

keyboard_controller = KeyboardController(
    vehicle=vehicle,
    keyboard_listener=keyboard_listener,
    action_space=[
        {"steering_angle": 0, "speed": 3, "index": 0},
        {"steering_angle": 30, "speed": 3, "index": 1},
        {"steering_angle": -30, "speed": 3, "index": 2},
        {"steering_angle": 0, "speed": -3, "index": 3},
    ],
    index_key_map = {
        0: "Key.up",
        1: "Key.left",
        2: "Key.right",
        3: "Key.down",
    },
    save=True,
    save_dir='saved_data/images',
    save_interval=0.2,
)

- run

In [ ]:
keyboard_controller.run(exit_key='Key.esc')

## 1.5. Auto Model Load Controller

- set controller

In [ ]:
from aicastle.deepracer.vehicle.api.controller import AutoModelLoadController

auto_model_load_controller = AutoModelLoadController(
    vehicle=vehicle,
    keyboard_listener=keyboard_listener,
    key_model_map={
        "1":"model1",
        "2":"model2",
    }
)

- run

In [ ]:
auto_model_load_controller.run(exit_key='Key.esc')

## 1.6. Model Analyize

- extract model_zip

In [ ]:
from aicastle.deepracer.vehicle.api.utils import extract_model

model_zip_path = "saved_data/models/model2.tar.gz"
model_folder_path = extract_model(model_zip_path)
print(model_folder_path)

- load analyzer

In [ ]:
from aicastle.deepracer.vehicle.api.analyzer import ModelAnalyzer

analyzer = ModelAnalyzer(model_folder_path)

- show heatmap

In [ ]:
import cv2
import time
from collections import deque
from IPython.display import clear_output
from aicastle.deepracer.vehicle.api.utils import read_image, show_image

heatmap_frame_queue = deque(maxlen=1)
show_image(frame_queue=heatmap_frame_queue, window_name="heatmap")

try :
    while True : 
        # image = read_image("sample/images/2_straight.png", color="bgr")
        image = vehicle.get_image()
        heatmap_img, pred, argmax_val = analyzer.visualize(image, alpha=0.4)
        heatmap_frame_queue.append(heatmap_img)
        print("Predictions:", pred.round(2))
        print("Argmax Value:", argmax_val)
        clear_output(wait=True)
        time.sleep(0.1)
except :
    pass